In [1]:
# fetch dataset name list
import os
hhm_path = 'data/hhblits_example/'
pdb_path = 'data/pdb_example/'
hhm_path_files = os.listdir(hhm_path)  
name_list = []
for fi in hhm_path_files: 
    hhm_name = fi.split('.')[0]
    name_list.append(hhm_name)
print(len(name_list))

1742


In [9]:
# one-hot dict for proteins
import numpy as np
protein_dict = {'C':np.eye(20)[0], 'D':np.eye(20)[1], 'S':np.eye(20)[1], 'Q':np.eye(20)[3], 'K':np.eye(20)[4],
        'I':np.eye(20)[5], 'P':np.eye(20)[6], 'T':np.eye(20)[7], 'F':np.eye(20)[8], 'N':np.eye(20)[9],
        'G':np.eye(20)[10], 'H':np.eye(20)[11], 'L':np.eye(20)[12], 'R':np.eye(20)[13], 'W':np.eye(20)[14],
        'A':np.eye(20)[15], 'V':np.eye(20)[16], 'E':np.eye(20)[17], 'Y':np.eye(20)[18], 'M':np.eye(20)[19]}
#print(protein_dict)

In [26]:
# generate 1024 cut-off dataset
np.set_printoptions(threshold=np.inf)
shhm_5A_mae_path = 'data/5A_mae_shhm/'
all_sample_list = []
with open("data/dataset_alphafold.txt") as file:
    line = file.readline()
    while line:
        if(line[0] == '>'):
            uniprot_id = line[1:].strip()
            seq = file.readline().strip()
            label = file.readline().strip()
            feature_matrix = np.zeros([1024, 52], float) # 20+30+1,onehot+spacehhblits+mask+label
            shhm_matrix = np.loadtxt(shhm_5A_mae_path + uniprot_id + ".shhm")
            # check seq length
            if(len(seq) <= 1024):
                for i in range(len(seq)):
                    feature_matrix[i,0:20] = protein_dict[seq[i]]
                    feature_matrix[i,20:50] = shhm_matrix[i,:]
                    feature_matrix[i,-2] = 1
                    feature_matrix[i,-1] = label[i]
            else: # cut off
                for i in range(1024):
                    feature_matrix[i,0:20] = protein_dict[seq[i]]
                    feature_matrix[i,20:50] = shhm_matrix[i,:]
                    feature_matrix[i,-2] = 1
                    feature_matrix[i,-1] = label[i]
        #print(feature_matrix.shape) #1024*52
        all_sample_list.append(feature_matrix)
        line = file.readline()
all_sample = np.array(all_sample_list)
print(all_sample.shape) # (1742, 1024, 52)
np.save("data/dataset_5A_mae_shhm_1024.npy", all_sample)

(1742, 1024, 52)


In [28]:
# load 1024 cut-off dataset
dataset = np.load("data/dataset_5A_mae_shhm_1024.npy")
print(dataset.shape) # (1742, 1024, 52)
# split dataset
train_set = dataset[0:1642]
valid_set = dataset[1642:1692]
test_set = dataset[1692:1742]

(1742, 1024, 52)


In [ ]:
# 1 test MLP

In [ ]:
# 2 test CNN

In [ ]:
# 3 test RNN

In [ ]:
# 4 test LSTM

In [ ]:
# 5 test BiLSTM

In [ ]:
# 6 test CNN+BiLSTM

In [ ]:
# 7 test Transformer